In [1]:
import random
from IPython.display import clear_output

In [2]:
class Card:
    def __init__(self, suit, number):
        self.suit = suit
        self.number = number
        
    def getSuit(self):
        return self.suit
    
    def getNumber(self):
        return self.number
    
    def __str__(self):
        return f"{self.number} of {self.suit}"
        
    

In [3]:
class DeckAndDiscardPile:
    '''This class handles the deck and a discard pile.'''
    suits = ["Clubs", "Spades", "Hearts", "Diamonds"]
    numbers = ["2","3","4","5","6","7","8","9","10","Jack","Queen","King","Ace"]
    
    
    def __init__(self):
        self.deckCards = []
        self.discardedCards = []
        for suit in DeckAndDiscardPile.suits:
            for number in DeckAndDiscardPile.numbers:
                self.deckCards.append(Card(suit,number))
        self.shuffle(includeMessage = False)
        
    def shuffle(self, includeMessage = True):
        '''Adds cards from the discard pile to the deck and shuffles the deck.'''
        
        self.deckCards.extend(self.discardedCards)
        
        # To shuffle the deck we will switch elements at random indices with each other many times
        for _ in range(10000):
            index1 = random.randint(0, len(self.deckCards) - 1)
            index2 = random.randint(0, len(self.deckCards) - 1)
            self.deckCards[index1], self.deckCards[index2] = self.deckCards[index2], self.deckCards[index1]
        
        if includeMessage:
            print("Deck was shuffled")
            input("Press ENTER to continue")
    
    def discardCards(self, cards=[]):
        '''Discards a list of cards'''
        for card in cards:
            # This prepends the cards to the list
            self.discardedCards.insert(0, card)
    
    def drawCard(self): 
        if len(self.deckCards) == 0:
            self.shuffle()
        if len(self.deckCards) > 0:          
            return self.deckCards.pop()
        else:
            self.shuffle()
        
    def view(self):
        for card in self.deckCards:
            print(f"{card} ")

In [4]:
class Hand:
    # Ace has value 11 here, even though it can have value 11 or 1. This is handled in the addCard method
    numberValues = {"2":2, "3":3, "4":4, "5":5, "6":6, "7":7, "8":8, "9":9, "10":10, "Jack":10, "Queen":10, "King":10, "Ace":11}
    
    def __init__(self):
        self.cards = []
        self.value = 0
        self.aceCounter = 0
        
    def depict(self, hideFirstCard):
        '''This method prints the hand to the console.'''
        # The first card might be hidden if it is the dealer's hand
        firstOpenCard = 0
        if(hideFirstCard):
            print("Card hidden")
            firstOpenCard = 1
        for i in range(firstOpenCard, len(self.cards)):
            print(self.cards[i])
    
    def addCard(self, card):
        self.cards.append(card)
        if card.getNumber() == "Ace":
            self.aceCounter += 1
        self.value += Hand.numberValues[card.getNumber()]
        # If we would have busted but have aces that are still worth 11, then we can decrease their value to 1. Thereby
        # lowering the total value by 10
        if self.value > 21 and self.aceCounter > 0:
            self.value -= 10
            self.aceCounter -= 1        
      
    def checkBlackjack(self):
        # For a hand to be a blackjack it need to consist of only an ace and a card worth 10 (i.e. a 10 or face card).
        # As far as I can tell, if the hand consists of exactly two cards and has a value of 21 then this must be the case.
        return len(self.cards) == 2 and self.value == 21
    
    def getCards(self):
        return self.cards
    
    def getValue(self):
        return self.value
    
    def reset(self):
        '''Returns old cards for discard pile and resets relevant attributes for next round.'''
        oldCards = self.cards
        self.cards = []
        self.value = 0
        self.aceCounter = 0
        return oldCards

In [5]:
class Player:
    def __init__(self, balance = 1000):
        self.balance = balance
        self.hand = Hand()
        self.betAmount = None
        
    def addCard(self, card):
        self.hand.addCard(card)
        
    def bet(self):
        '''Used to get a valid bet from the player and set the betAmount variable'''
        
        # helper function to avoid Type- and ValueErrors
        def readInt(message=""):
            while True:
                try:
                    inputInt = int(input(message))
                except ValueError or TypeError:
                    print("Please enter an integer.")
                else:
                    return inputInt
        
        print(f"You balance is: {self.balance}")
        
        validBet = False
        while not validBet:
            amount = readInt("Amount to bet: ")
            if 0 < amount <= self.balance:
                self.balance -= amount
                self.betAmount = amount
                validBet = True
            elif amount > self.balance:
                print("That is more than you have.")
            elif amount == 0:
                print("You have to bet more than 0.")
            else: 
                print("Don't try to cheat!")
        clear_output(wait=False)

                
    def dealerWin(self):
        print("The house wins. You loose your bet.")
        # We don't need to subtract from balace because this was already done with the bet method
        
    def playerWin(self):
        self.balance += 2*self.betAmount
        print("You win! You get double your bet back!")
    
    def dealerBust(self):
        self.balance += 2*self.betAmount
        print("The dealer busted. You get double your bet back!")
        
    def bust(self):
        print("You busted. You loose your bet!")
        # We don't need to subtract from balace because this was already done with the bet method
    
    def push(self):
        self.balance += self.betAmount
        print("Push. You get your bet back.")
        
    def blackjackPush(self):
        self.balance += self.betAmount
        print("Two blackjacks! A push!")
    
    def blackjackPlayerWin(self):
        # If multiplying by 2.5 would lead to an uneven number, then we would want to round up. I want the balance to 
        # stay a natural number.
        self.balance = int(self.balance + 2.5*self.betAmount)
        if self.betAmount % 2 != 0:
            self.balance += 1
        print("Blackjack! You get 2.5 times your bet back!")
        
    def blackjackDealerWin(self):
        print("The dealer has a blackjack. You loose your bet.")
    

In [6]:
class Dealer:
    def __init__(self):
        self.hand = Hand()
        
    def addCard(self, card):
        self.hand.addCard(card)  
    

In [7]:
def depictTable(hideFirstDealerCard):
    print('==========')
    dealer.hand.depict(hideFirstCard = hideFirstDealerCard)
    print('DEALER HAND')
    print()
    print(f'bet: {player.betAmount}')
    print()
    print('YOUR HAND')
    player.hand.depict(hideFirstCard = False)      
    print('==========')

In [8]:
# Main loop
playRound = True
player = Player()
dealer = Dealer()
deckAndPile = DeckAndDiscardPile()

print(
'''
Welcome to Blackjack!

To start press ENTER
''')
input() # This lets the program wait until the user presses a key
clear_output(wait=False)
   
while(playRound):
    deckAndPile.discardCards(player.hand.reset())
    deckAndPile.discardCards(dealer.hand.reset())

    # The player places his bet
    player.bet()        

    # The player and dealer receive their cards.
    for _ in range(2):
        player.addCard(deckAndPile.drawCard())
        dealer.addCard(deckAndPile.drawCard())

    # Check for blackjacks
    if dealer.hand.checkBlackjack() and player.hand.checkBlackjack():
        depictTable(hideFirstDealerCard = False)
        player.blackjackPush()
    elif player.hand.checkBlackjack():
        depictTable(hideFirstDealerCard = False)
        player.blackjackPlayerWin()
    elif dealer.hand.checkBlackjack():
        depictTable(hideFirstDealerCard = False)
        player.blackjackDealerWin()
    else:    
        # The hands are shown
        depictTable(hideFirstDealerCard = True)

        # Player decides between draw or stand
        drawOrStand = None
        while not(drawOrStand == "d" or drawOrStand == "s"):
            drawOrStand = input("Enter s to stand or d to draw: ")
        clear_output(wait=False)

        # As long as the player wants to draw more cards we will execute this
        playerBust = False
        dealerPlays = True
        while not playerBust and drawOrStand == "d":
            player.addCard(deckAndPile.drawCard())
            if player.hand.value > 21:
                # The player busts. We directly depict the table with the new card and the visible dealer card.
                depictTable(hideFirstDealerCard = False)
                player.bust()
                playerBust = True
            else:                
                depictTable(hideFirstDealerCard = True)
                # We ask the player what he wants to do next
                drawOrStand = None
                while(not(drawOrStand == "d" or drawOrStand == "s")):
                    drawOrStand = input("Enter s to stand or d to draw: ")
                clear_output(wait=False)

        # If we are here and the player hasn't busted, then he is done with drawing and it is the dealer's turn
        if not playerBust:
            depictTable(hideFirstDealerCard = False)
            input("Press ENTER to continue")
            clear_output(wait=False)

            while dealerPlays:           
                # The dealer is supposed to draw cards as long as he is below 17
                if dealer.hand.value < 17:
                    dealer.addCard(deckAndPile.drawCard())
                    depictTable(hideFirstDealerCard = False)
                    input("Press ENTER to continue")
                    clear_output(wait=False)
                elif 17 <= dealer.hand.value <= 21:
                    dealerPlays = False
                elif 21 < dealer.hand.value:
                    player.dealerBust()
                    dealerPlays = False

            # Here we check for the remaining outcomes in case the player and dealer haven't busted
            if player.hand.value < dealer.hand.value <= 21:
                player.dealerWin()
            elif dealer.hand.value < player.hand.value <= 21:
                player.playerWin()
            elif player.hand.value == dealer.hand.value and dealer.hand.value <= 21:
                player.push()

    # Here the round is over
    print(f"Your balance is {player.balance}")
    if player.balance > 0:
        anotherRound = None
        while not(anotherRound == "y" or anotherRound == "n"):
            anotherRound = input("Would you like to play another round (y/n)? ")
        if anotherRound == "n":
            playRound = False
            print("We are looking forward to your next visit!")
    else:
        print("You have no more money left. We are throwing you out of the casino!")
        playRound = False

Ace of Spades
Queen of Diamonds
DEALER HAND

bet: 200

YOUR HAND
3 of Hearts
Jack of Clubs
The dealer has a blackjack. You loose your bet.
Your balance is 910
Would you like to play another round (y/n)? n
We are looking forward to your next visit!
